In [3]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.2136, 0.6029, 0.7014],
        [0.8107, 0.1225, 0.3021],
        [0.4463, 0.6701, 0.9434],
        [0.9879, 0.8220, 0.3606],
        [0.5320, 0.2842, 0.3815]])


In [4]:
# testing lib fn
from __future__ import print_function, unicode_literals
from unicodedata import normalize
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Masking
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
df = pd.read_pickle('df1_pos_W.pkl')

In [6]:
df.head()

,context_lemma_pos,question_lemma_pos,answers_lemma_pos,answer_len,question_len,context_len
2,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",1,8,75
11,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyoncé, NN), (release, NN), (d...","[(2003, CD)]",1,5,75
12,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(how, WRB), (many, JJ), (grammy, JJ), (award,...","[(five, CD)]",1,9,75
15,"[(follow, VBG), (disbandment, NN), (destiny, N...","[(after, IN), (second, JJ), (solo, NN), (album...","[(act, VBG)]",1,8,112
17,"[(follow, VBG), (disbandment, NN), (destiny, N...","[(to, TO), (set, VB), (record, NN), (grammys, ...","[(six, CD)]",1,7,112


In [7]:
len(df)

31989

In [11]:
df.question_lemma_pos

2         [(when, WRB), (beyonce, NN), (leave, VBP), (de...
11        [(when, WRB), (beyoncé, NN), (release, NN), (d...
12        [(how, WRB), (many, JJ), (grammy, JJ), (award,...
15        [(after, IN), (second, JJ), (solo, NN), (album...
17        [(to, TO), (set, VB), (record, NN), (grammys, ...
                                ...                        
130044    [(from, IN), (city, NN), (arkefly, NN), (offer...
130046    [(in, IN), (us, PRP), (state, NN), (kathmandu,...
130047    [(what, WP), (yangon, VBZ), (previously, RB), ...
130048    [(with, IN), (belorussian, JJ), (city, NN), (k...
130049    [(in, IN), (year, NN), (kathmandu, NN), (creat...
Name: question_lemma_pos, Length: 31989, dtype: object

In [12]:
pos_dict=lib.get_pos_dict(df,columns=['context_lemma_pos', 'question_lemma_pos'])
pos_dict

{'': 0.0,
 'NN': 0.9396290695832222,
 'VBP': 0.6481191750452352,
 'JJ': 0.05336812937575197,
 'CD': 0.3325599466223039,
 'NNS': 0.1908087792008828,
 'VBD': 0.9282145062504379,
 'VBG': 0.5582149791824439,
 'RB': 0.9694721332405776,
 'VBZ': 0.2231554698427548,
 'VBN': 0.6755873747134403,
 'PRP$': 0.7388765498478684,
 'VB': 0.010894167732162674,
 'DT': 0.7572363251420033,
 'PRP': 0.489629885242439,
 'JJR': 0.264321492287111,
 'IN': 0.05726307181474455,
 'JJS': 0.1943286222958902,
 'NNP': 0.4110884066302938,
 'MD': 0.19010961518913128,
 'WDT': 0.25797421394306674,
 'FW': 0.2029404303436384,
 '$': 0.7207393953877647,
 'WRB': 0.15018010009942107,
 'RP': 0.3412654226213009,
 'RBR': 0.7220541548278566,
 'TO': 0.38743332497416905,
 'RBS': 0.5365336483567881,
 'CC': 0.9896245981926637,
 'WP$': 0.0886279399098866,
 'WP': 0.7261817587059886,
 'EX': 0.3419149946459943,
 'NNPS': 0.6571860596137962,
 'PDT': 0.3730334532730085,
 'SYM': 0.21167556750789263,
 'POS': 0.9148450236064504,
 'UH': 0.51130199